In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import re
import pandas as pd

In [2]:
def ex_tag(sid, page):
    ### 뉴스 분야(sid)와 페이지(page)를 입력하면 그에 대한 링크들을 리스트로 추출하는 함수 ###
    
    ## 1.
    url = f"https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={sid}"\
    "#&date=%2000:00:00&page={page}"
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    a_tag = soup.find_all("a")
    
    ## 2.
    tag_lst = []
    for a in a_tag:
        if "href" in a.attrs:  # href가 있는것만 고르는 것
            if (f"sid={sid}" in a["href"]) and ("article" in a["href"]):
                tag_lst.append(a["href"])
                
    return tag_lst

In [3]:
def re_tag(sid):
    ### 특정 분야의 100페이지까지의 뉴스의 링크를 수집하여 중복 제거한 리스트로 변환하는 함수 ###
    re_lst = []
    for i in tqdm(range(100)):
        lst = ex_tag(sid, i+1)
        re_lst.extend(lst)

    # 중복 제거
    re_set = set(re_lst)
    re_lst = list(re_set)
    
    return re_lst

In [4]:
all_hrefs = {}
sids = [i for i in [104, 101, 102, 103, 105, 107, 100]]  # 분야 리스트

# 각 분야별로 링크 수집해서 딕셔너리에 저장
for sid in sids:
    sid_data = re_tag(sid)
    all_hrefs[sid] = sid_data

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [33]:
def art_crawl(all_hrefs, sid, index):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수 
    
    Args: 
        all_hrefs(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [100: 정치, 101: 경제, 102: 사회, 103: 생활/문화, 104: 세계, 105: IT/과학]
        index(int): 링크의 인덱스
    
    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리
    
    """
    art_dic = {}
    
    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"
    
    url = all_hrefs[sid][index]
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    
    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)
    
    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)
    
    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)
    
    ## 특수문자
    #pattern = r'[^a-zA-Z가-힣0-9]'
    #title_str = re.sub(pattern=pattern, repl=' ', string=title_str)
    
    ## 3.
    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str
    art_dic['target'] = {104:0, 101:1, 102:2, 103:3, 105:4, 107:5, 100:6}[sid]
    
    
    return art_dic

In [34]:
# 모든 섹션의 데이터 수집 (제목, 날짜, 본문, section, url)
section_lst = [s for s in [104, 101, 102, 103, 105, 107, 100]]
artdic_lst = []

for section in tqdm(section_lst):
    for i in tqdm(range(len(all_hrefs[section]))):
        art_dic = art_crawl(all_hrefs, section, i)
        art_dic["section"] = section
        art_dic["url"] = all_hrefs[section][i]
        artdic_lst.append(art_dic)

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/358 [00:00<?, ?it/s]

  0%|          | 0/579 [00:00<?, ?it/s]

  0%|          | 0/629 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/156 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/617 [00:00<?, ?it/s]

In [35]:
art_df = pd.DataFrame(artdic_lst)
art_df.to_csv("article_df.csv")

In [36]:
art_df['ID'] = [f'naver_{(x)}' for x in art_df.index]
art_df = art_df.rename(columns={'title':'text'})

data_raw = pd.read_csv('data/train.csv')
data_augmentation = pd.concat([data_raw,art_df[['ID','text','target','url','date']]]).reset_index(drop=True)
data_augmentation.to_csv('data/train_augmentation.csv')

In [37]:
art_df

,text,date,main,target,section,url,ID
0,유조선 공격에 미군 사망까지…중동 긴장 고조에 유가 '출렁',2024.01.29. 오후 4:37,국제 유가 오전 한때 1.5%가량 뛰었다 상승폭 축소\n\n\n\n요르단내 미군 주...,0,104,https://n.news.naver.com/mnews/article/001/001...,naver_0
1,"외교장관 임명 19일 지나도 통화없는 한중…중국 ""소통 유지 용의""",2024.01.29. 오후 5:58,▲ 23일 가미카와 요코 일본 외무상과 첫 통화하는 조태열 외교부 장관중국 외교부가...,0,104,https://n.news.naver.com/mnews/article/055/000...,naver_1
2,"""파리 봉쇄"" 농민들 위협에 佛내무부, 파리주변 보안군 대거 배치",2024.01.29. 오후 5:47,"농민 트랙터들, 파리 집결 시작…파리 공항 등 봉쇄 위협내무장관, 파리로 향하는 8...",0,104,https://n.news.naver.com/mnews/article/003/001...,naver_2
3,"""대만을 국가로 언급해서""…中, 한국 LoL 중계 6년만 중단",2024.01.29. 오전 10:12,"홍콩매체 ""한국팀 젠지, 대만을 국가로 언급한 사건과 관련""\n\n\n\n2024 ...",0,104,https://n.news.naver.com/mnews/article/001/001...,naver_3
4,"백악관 에너지안보 고문 ""후티 공격 인플레 영향 제한적""",2024.01.29. 오전 10:33,예멘 남쪽 아덴항과 아덴만 해상 [구글지도 캡처. 재판매 및 DB 금지] (서...,0,104,https://n.news.naver.com/mnews/article/001/001...,naver_4
...,...,...,...,...,...,...,...
2392,"이준석 ""여성도 軍복무해야 경찰관·소방관 지원 가능하게""(종합)",2024.01.29. 오전 10:46,"'여성희망 복무제' 2030년 도입 공약…""간부·부사관 아닌 일반병사 근무""군자녀 ...",6,100,https://n.news.naver.com/mnews/article/001/001...,naver_2392
2393,"민주당, 경찰대 출신 '미니스커트 여경'·초등교사노조 간부 영입",2024.01.29. 오후 12:40,"이지은 전 총경, 백승아 전 교사 각각 11, 12호 인재로 영입\n\n\n\n더불...",6,100,https://n.news.naver.com/mnews/article/088/000...,naver_2393
2394,이준석표 국방정책… “여성도 軍복무해야 경찰·소방 지원가능”,2024.01.29. 오전 11:17,"‘女 신규공무원 병역 의무화’ 이르면 2030년 도입“시민 절반만 국방의무 부담, ...",6,100,https://n.news.naver.com/mnews/article/366/000...,naver_2394
2395,당대표 이준석-원내대표 양향자...개혁신당 신설합당,2024.01.29. 오전 10:40,슬로건은 '한국의희망'여성 공무원 군복무 의무화 공약 발표 \n\n\n\n이준석 개...,6,100,https://n.news.naver.com/mnews/article/014/000...,naver_2395


In [13]:
import pandas as pd
df__ = pd.read_csv('../article_df.csv')

In [15]:
df__.value_counts('target')

target
2    629
6    617
1    579
0    358
4    156
3     58
Name: count, dtype: int64